<a href="https://colab.research.google.com/github/saksham-giri/SIG-Python-Project/blob/main/Pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p ./dataset --unzip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:26<00:00, 162MB/s]
100% 2.29G/2.29G [00:26<00:00, 92.0MB/s]


In [21]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [25]:
# Define dataset paths
base_dir = './dataset/chest_xray'  # Adjust this to match your dataset's location
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [26]:
# Step 4: Data Augmentation and Rescaling
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [28]:
# Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [29]:
# Compile the Model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [30]:
# Train the Model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 338s 2s/step - accuracy: 0.7479 - loss: 0.5849 - val_accuracy: 0.5625 - val_loss: 0.8954
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


163/163 ━━━━━━━━━━━━━━━━━━━━ 329s 2s/step - accuracy: 0.8450 - loss: 0.3551 - val_accuracy: 0.6250 - val_loss: 0.8557
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 95us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.8602 - loss: 0.3295 - val_accuracy: 0.8125 - val_loss: 0.4200
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 104us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 368s 2s/step - accuracy: 0.8839 - loss: 0.2750 - val_accuracy: 0.8750 - val_loss: 0.4398
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.8895 - loss: 0.2754 - val_accuracy: 0.7500 - val_loss: 0.5961
Epoch 10/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 164us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [31]:
# Save the Model
model.save('./pneumonia_model.h5')

In [32]:
# Function to preprocess a single image for prediction
def preprocess_image(img_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to match the model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values
    return img_array

In [33]:
# Function to predict pneumonia from an uploaded image
def predict_pneumonia(img_path):
    processed_image = preprocess_image(img_path)
    prediction = model.predict(processed_image)
    return "Yes" if prediction[0][0] > 0.5 else "No"

In [34]:
# Example usage
try:
    # Replace 'example_image.jpg' with the path of an image you want to test
    test_image_path = './example_image.jpg'  # Path to your test image
    result = predict_pneumonia(test_image_path)
    print(f"Does the X-ray indicate pneumonia? {result}")
except Exception as e:
    print(f"Error during prediction: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Does the X-ray indicate pneumonia? No
